In [ ]:
# this part of code is for data prepare
import scipy.io as sio  
import matplotlib.pyplot as plt  
import numpy as np  

# read the train data
matfn=u'train_32x32.mat'  
data=sio.loadmat(matfn)  
orign_X=data['X']
y_train=data['y']
# change the shape of x_train
x_train= np.transpose(orign_X, (3,0,1,2 ))
y_train[y_train==10] = 0

# read the test data
matfn=u'test_32x32.mat'  
data=sio.loadmat(matfn)  
orign_X=data['X']
# change the shape of x_train
x_test= np.transpose(orign_X, (3,0,1,2 ))
y_test=data['y']
y_test[y_test==10] = 0

import numpy as np
from keras.layers import Input, Dense,normalization
from keras.models import Model
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


# define the batch_size,num_classes
batch_size = 32
num_classes = 10

# Instead of running the whole data part of the data can be used 
# defalut set the whole train data and test data
num_train = 73257
num_test = 26032 
num_val = 26032 

x_train_cnn = x_train[:num_train,:,:,:]
y_train_cnn = y_train[:num_train]
x_test_cnn = x_test[:num_test,:,:,:]
y_test_cnn = y_test[:num_test]
x_val_cnn = x_test[:num_val,:,:,:]
y_val_cnn = y_test[:num_val]

x_train_cnn = x_train_cnn.astype('float32')
x_test_cnn = x_test_cnn.astype('float32')
x_val_cnn = x_test_cnn.astype('float32')
x_train_cnn /= 255.
x_test_cnn /= 255.
x_val_cnn /= 255.

# Convert class vectors to binary class matrices to reduce runnning time
y_train_cnn = keras.utils.to_categorical(y_train_cnn, num_classes)
y_test_cnn = keras.utils.to_categorical(y_test_cnn, num_classes)
y_val_cnn = keras.utils.to_categorical(y_val_cnn, num_classes)





In [ ]:
# this part of code used to build the nerual network and fit the train data

# Create CNN topology
model = Sequential()

model.add(normalization.BatchNormalization(input_shape=x_train_cnn.shape[1:]))

model.add(Conv2D(16, (9, 9),strides=3,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))
model.add(normalization.BatchNormalization())

model.add(Conv2D(32, (3, 3),strides=1,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))
model.add(normalization.BatchNormalization())


model.add(Conv2D(32, (3, 3),strides=1,padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3),strides=1,padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3),strides=1, padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=1))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# Fit model to data
epochs = 4
model.fit(x_train_cnn, y_train_cnn, batch_size=batch_size, epochs=epochs,validation_data=(x_val_cnn, y_val_cnn), shuffle=True)




In [ ]:
# this part of code used to predict the testdata
# output the confusion_matrix,Accuracy,P,R and F values for the testdata

import matplotlib.pyplot as plt
!pip install prettytable
from prettytable import PrettyTable


from sklearn.metrics import confusion_matrix
import itertools


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.figure()
    plt.show()


def resultanalysis(result_list,y_vetor):
        length=len(y_vetor)
        result=result_list
        result_matrix=np.zeros((length, 1))
        for i in range(0,length):
             result_matrix[i]=np.where(result[i]==np.max(result[i]))[0][0]
        y_matrix=np.zeros((length, 1))
        for i in range(0,length):
             y_matrix[i]=np.where(y_vetor[i]==np.max(y_vetor[i]))[0][0]
        result_matrix=np.hstack([y_matrix,result_matrix]).astype('int32')        
        confusionlist=np.zeros([10,10],np.int16)
        for item in result_matrix:
            confusionlist[item[0]][item[1]]+=1

        labels=np.arange(10)
        plot_confusion_matrix(confusionlist,labels)
        right_number=0
        for i in range(0,10):
            right_number+=confusionlist[i][i]
        # use confusionlist calculate accurcy   
        accurcy=float(right_number)/length
        print("The test accurcy is "+str(accurcy))
        Precision=np.zeros([10])
        Recall=np.zeros([10])
        F1=np.zeros([10])
        for k in range(0,10):
            Precision[k]=round(float(confusionlist[k][k])*100/confusionlist.sum(axis=1)[k],2)
            Recall[k]=round(float(confusionlist[k][k])*100/confusionlist.sum(axis=0)[k],2)
            F1[k]=round(2*(Recall[k]*Precision[k])/(Recall[k]+Precision[k]),2)
        x = PrettyTable(['Label', 'Precision', 'recall', 'F1-score','Support'])
        for i in range(0,10):
             x.add_row([labels[i],Precision[i],Recall[i], F1[i],confusionlist.sum(axis=1)[i]])
        print('Table 1. Precision,Recall,F1 and Support score for each label')
        print(x)
resulttest=model.predict(x_test_cnn)
resultanalysis(resulttest,y_test_cnn)

In [ ]:
# This two functions are used to test the preformance of the classifier using the 10-fold cross validation
# after running the code above simple run this part 
# output the Accuracy,P,R and F values 10 times for every fold respctively
# This will cost a long time
def tenFloder(X,y):
    arraylist =np.arange(X.shape[0])
    np.random.shuffle(arraylist)
    randomlist=np.array_split(arraylist,10)
    accuracysum=0
    RPFtotallist=[]
    for item in randomlist:
        traningindex=np.setdiff1d(arraylist,item)
        test_X=X[item]
        test_y=y[item]
        traning_X=X[traningindex]
        traning_Y=y[traningindex]
        result=train_predictfunction(traning_X,traning_Y,test_X,test_y)
def train_predictfunction(x_train,y_train,x_val,y_val):
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    epochs = 4
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val, y_val), shuffle=True)
    result=model.predict(x_val)
    resultanalysis(result,y_val)
    
tenFloder(x_train_cnn, y_train_cnn)